# Programming Assignment: Social Network Database Fix

## Welcome!

You've reached a practical part of our course, **Generative AI for Software Development - AI-Powered Software and System Design**. This task is all about diving into a database designed for managing a social network. It's a hands-on challenge to test your skills and see how well you can work with AI tools, like a Language Learning Model (LLM).

## Your Mission

Your job is to use an LLM to understand the code for creating a database and query over it.

### Tasks:

1. **Write Three Functions:** Use the LLM to help you craft three functions. They should:
   - List all members of a specific club.
   - List all friends of a specific person.
   - List all people who consider a specific person their friend.

### Working with the LLM:

- **Ask the LLM for Help:** Use the LLM to guide you through fixing the database and creating your functions.
- **Use Its Advice Wisely:** Remember, the LLM's advice might not always be spot-on. It's up to you to decide what's useful.

## What to Submit:

Please submit this Jupyter notebook containing:
- The three functions you've written.

## How We'll Grade It:

- **Functionality of Your Functions:** We'll see if each of your functions is doing exactly what it's supposed to do.

## Tips for Success:

- **Be Clear with Your Questions:** The clearer your questions to the LLM, the better help you'll get.
- **Test the Advice:** Always test out the LLM’s suggestions to ensure they work as expected.
- **Hints**: If you struggle with this assignment, you can check some hints we left to you in the bottom of the assignment!

### Necessary imports

In [1]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import numpy as np
import os

In [2]:
import unittests

## The main code

Below is the primary code for the assignment. It will populate the tables you will be using in the assignment. There is no need to change this code and changing it may result in incorrect results. 

**If you need to start over with a clean version of this assignment, there's a folder named `backup_data` where you can find a fresh copy.**

In [3]:
def load_dataset(path = "./"):
    """Loads the dataset
    
    RUN THE FUNCTION WITHOUT ANY PARAMETER. 
    THE PARAMETER IS FOR GRADING PURPOSES ONLY.
    
    """
    ## DO NOT CHANGE THIS PART ##
    # To ensure reproducibility
    
    np.random.seed(42)
    
    ## DO NOT CHANGE THIS PART ##
    # To avoid populating the same database several times, always exclude it between a new execution.
    if 'social_network.db' in os.listdir(path):
        os.remove('social_network.db')
    
    # You may pass echo=True to output information that can help you
    engine = create_engine(f'sqlite:///{os.path.join(path, "social_network.db")}', echo=False)
    Base = declarative_base()
    
    # Define the friendship association table
    friendships = Table('friendships', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id'), primary_key=True),
        Column('friend_id', Integer, ForeignKey('people.id'), primary_key=True)
    )
    
    class Person(Base):
        __tablename__ = 'people'
        id = Column(Integer, primary_key=True)
        name = Column(String, primary_key = False)
        age = Column(Integer)
        gender = Column(String)
        location = Column(String)
    
        friends = relationship("Person",
                               secondary=friendships,
                               primaryjoin=(friendships.c.person_id == id),
                               secondaryjoin=(friendships.c.friend_id == id),
                               backref="friend_of")
        clubs = relationship("Club", secondary="club_members")
    
    class Club(Base):
        __tablename__ = 'clubs'
        id = Column(Integer, primary_key=True)
        description = Column(String)
    
        members = relationship("Person", secondary="club_members")
    
    club_members = Table('club_members', Base.metadata,
        Column('person_id', Integer, ForeignKey('people.id')),
        Column('club_id', Integer, ForeignKey('clubs.id'))
    )
    
    # Create the tables in the database
    Base.metadata.create_all(engine)
    
    # Create a session to interact with the database
    Session = sessionmaker(bind=engine)
    session = Session()
    
    # Sample data
    names = [("Alice", "New York", "Non-binary", 30), 
             ("Bob", "Los Angeles", "Male", 18), 
             ("Charlie", "Chicago", "Male", 60), 
             ("David", "Houston", "Male", 59),
             ("Eve", "Phoenix", "Non-binary", 18), 
             ("Frank", "Los Angeles", "Non-binary", 72), 
             ("Grace", "Chicago", "Female", 35), 
             ("Henry", "Houston", "Male", 21), 
             ("Ivy", "New York", "Female", 46), 
             ("Elena", "Phoenix", "Female", 66)]
    club_descriptions = [
        "Book Club", "Hiking Club", "Chess Club", "Photography Club", "Cooking Club",
        "Music Club", "Gaming Club", "Fitness Club", "Art Club", "Travel Club"
    ]

    
    
   # Populate People table
    people = []
    for person in names:
        person = Person(
            name=person[0],
            age=person[-1],
            gender=person[2],
            location=person[1]
        )
        people.append(person)
        session.add(person)
    
    # Populate Friendships
    for _ in range(200):  # Create 200 random friendships
        person1 = np.random.choice(people)
        person2 = np.random.choice(people)
        if person1 != person2 and person2 not in person1.friends:
            person1.friends.append(person2)
            session.flush()
    
    # Populate Clubs table
    clubs = []
    for description in club_descriptions:
        club = Club(description=description)
        clubs.append(club)
        session.add(club)
    
    # Populate Club Members table
    for club in clubs:
        num_members = np.random.randint(5, 10)
        members = np.random.choice(people, num_members)
        club.members.extend(members)
    
    # Commit the changes
    session.commit()
    
    # Close the session
    session.close()
    
    print("Database created and populated successfully!")
    return session, Club, Person, friendships

In [4]:
# Loads the dataset
session, Club, Person, friendships = load_dataset()

Database created and populated successfully!


/var/folders/0n/m62rpbbj60b2p0xcy2j637800000gn/T/ipykernel_54812/2671118863.py:20: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()
/var/folders/0n/m62rpbbj60b2p0xcy2j637800000gn/T/ipykernel_54812/2671118863.py:83: SAWarning: relationship 'Club.members' will copy column clubs.id to column club_members.club_id, which conflicts with relationship(s): 'Person.clubs' (copies clubs.id to club_members.club_id). If this is not the intention, consider if these relationships should be linked with back_populates, or if viewonly=True should be applied to one or more if they are read-only. For the less common case that foreign key constraints are partially overlapping, the orm.foreign() annotation can be used to isolate the columns that should be written towards.   To silence this warning, add the parameter 'overlaps="clubs"' t

### Exercise 1

For this exercise, you are tasked with creating a function named `get_club_members`. This function should accept a description of a club and a session, and return a list of all its members. **Ensure that this function returns a list containing the defined Person objects.** It must input only a **club description**.

In [5]:
def get_club_members(session,club_description):
    """
    Returns a list of Person objects who are members of a club given the club's description.
    
    Parameters:
    - club_description (str): The description of the club for which members are to be retrieved.
    
    Returns:
    - List[Person]: A list of Person objects who are members of the specified club.
    """
    # Query the club based on the description
    club = session.query(Club).filter(Club.description == club_description).first()

    # If the club exists, return its members
    if club:
        return club.members
    return []  # Return an empty list if the club does not exist

In [6]:
# Example usage of the get_club_members function

# Assume the session and all models have been correctly set up and populated as per your initial code

# Fetching members of the "Hiking Club"
hiking_club_members = get_club_members(session,"Hiking Club")

# Printing out the names of all members of the Hiking Club
print("Members of the Hiking Club:")
for person in hiking_club_members:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

Members of the Hiking Club:
- Frank, Age: 72, Location: Los Angeles
- Ivy, Age: 46, Location: New York
- Eve, Age: 18, Location: Phoenix
- Alice, Age: 30, Location: New York
- David, Age: 59, Location: Houston
- Elena, Age: 66, Location: Phoenix


In [7]:
unittests.test_get_club_members(load_dataset, get_club_members)

 All tests passed!


### Exercise 2

In this exercise, you are required to create a function named `get_friends_of_person`. This function should accept the name of a person and a session, return a list of all their friends. **Ensure that this function returns a list containing the defined Person objects.** The input must be only the **name of a person**.

In [8]:
def get_friends_of_person(session, person_name):
    """
    Returns a list of Person objects who are friends with the specified person.
    
    Parameters:
    - person_name (str): The name of the person for whom to retrieve friends.
    
    Returns:
    - List[Person]: A list of Person objects who are friends with the specified person.
    """
    # Query the person based on the name
    person = session.query(Person).filter(Person.name == person_name).first()

    # If the person exists, return their friends
    if person:
        return person.friends
    return []  # Return an empty list if the person does not exist

In [9]:
# Example usage of the get_friends_of_person function

# Fetching friends of given name
name = "Bob"

alice_friends = get_friends_of_person(session,name)

# Printing out the names of all friends of Alice
print(f"Friends of {name}:")
for friend in alice_friends:
    print(f"- {friend.name}, Age: {friend.age}, Location: {friend.location}")

Friends of Bob:
- Alice, Age: 30, Location: New York
- David, Age: 59, Location: Houston
- Eve, Age: 18, Location: Phoenix
- Frank, Age: 72, Location: Los Angeles
- Henry, Age: 21, Location: Houston
- Ivy, Age: 46, Location: New York
- Elena, Age: 66, Location: Phoenix


In [10]:
unittests.test_get_friends_of_person(load_dataset, get_friends_of_person)

 All tests passed!


### Exercise 3

In this exercise, you're tasked with crafting a function called `get_persons_who_consider_them_friend`. This function should take two parameters: the name of an individual and a session. It will return a list of people who count this individual as a friend. It's important to remember that in our database, friendship isn't necessarily mutual. For example, Alice might consider Bob a friend, but Bob might not feel the same way about Alice. **Your function must return a list of Person objects for everyone who considers the input name their friend.** The input to this function should strictly be the **name of the person** you're inquiring about.

In [14]:
def get_persons_who_consider_them_friend(session, person_name):
    """
    Returns a list of Person objects who consider the specified person as their friend,
    in a scenario where friendships are unidirectional.
    
    Parameters:
    - person_name (str): The name of the person to find who is considered as a friend by others.
    
    Returns:
    - List[Person]: A list of Person objects who consider the specified person as their friend.
    """
    # Query the person based on the name
    person = session.query(Person).filter(Person.name == person_name).first()

    # If the person exists, find all people who have this person as a friend
    if person:
        # Check all persons to see who considers the queried person as a friend
        return [p for p in session.query(Person).all() if person in p.friends]

    # Return an empty list if the person does not exist
    return []  # Ensure this is reached if person is not found

In [15]:
# Example usage of the get_persons_who_consider_them_friend function

# Fetching people who consider given name as their friend
name = 'Bob'

name_friend_of = get_persons_who_consider_them_friend(session, name)

# Printing out the names of all people who consider Alice as their friend
print(f"People who consider {name} as their friend:")
for person in name_friend_of:
    print(f"- {person.name}, Age: {person.age}, Location: {person.location}")

People who consider Bob as their friend:
- Alice, Age: 30, Location: New York
- Charlie, Age: 60, Location: Chicago
- David, Age: 59, Location: Houston
- Frank, Age: 72, Location: Los Angeles
- Grace, Age: 35, Location: Chicago
- Henry, Age: 21, Location: Houston
- Ivy, Age: 46, Location: New York


In [16]:
unittests.test_get_persons_who_consider_them_friend(load_dataset, get_persons_who_consider_them_friend)

 All tests passed!


## Now test your Dataset!

<details>    
<summary>
    <font size="3" color="darkgreen"><b>Hint 1</b></font>
</summary>
<p>
Keep in mind that friendships are not always reciprocal. Be sure to mention this when requesting the LLM to write your last function.
</p>
</details>

Congratulations! You have finished the assignment! Keep it up!